In [ ]:
import numpy as np
import pandas as pd

from scipy.stats import norm
from scipy.optimize import minimize
from math import ceil

import warnings
warnings.filterwarnings('ignore')

In [ ]:
class Option():
    def __init__(self, price, strike, sigma , start, end, call = True, EU = True, riskfree=0.075, divs = True, n=10):
        
        self.asset_price = price
        
        self.strike = strike
        
        self.sigma = sigma
        
        self.EU = EU
        
        self.riskfree = riskfree
        
        self.__call = call
        
        self.__n = n 
        
        self.__divs = divs 
        
        self.start = pd.to_datetime(start, dayfirst=True)
        self.end = pd.to_datetime(end, dayfirst=True)
        self.days = (self.end - self.start).days
        self.T = self.days/365
        self.dt = self.T/n
    

    
    def grow_tree(self, n_steps = None):
        
        """"
        
        Creates binomial tree in matrix form: from initial value - step to the RIGHT = step up; step DOWN = step down
        n_steps: number of steps for a tree
        opt_price: if False, returns tree only for the asset price; if True returns asset price tree AND option price tree
        
        """   
        if n_steps == None:
            n_steps = self.__n
        
        self.up = np.exp(self.sigma*np.sqrt(self.dt))
        self.down = 1/self.up 
        
        self.growth_factor = 1 if self.__divs == False else np.exp(self.riskfree*self.dt)
        self.p = (self.growth_factor-self.down)/(self.up-self.down)
        self.q = 1 - self.p
        
        ###########  Ценовое дерево ############

        first_row = [self.asset_price]

        for i in range(n_steps):
            first_row.append(first_row[-1]*self.up)

        tree = np.array(first_row).reshape(1, n_steps+1)

        for i in range(1, n_steps+1):
            temp = np.append(tree[-1, :-i]*self.down, [np.NaN]*i)
            tree = np.vstack([tree, temp])

        self.tree = pd.DataFrame(tree)

        price_diff = self.tree - self.strike if self.__call == True else self.strike - self.tree
        price_diff = np.where(price_diff > 0, price_diff, 0)
        
        ###########  Опционное дерево ############
            
        last_col = pd.DataFrame(np.diagonal(np.fliplr(price_diff)))

        for i in range(1, len(last_col)):
            last_col[i] = (last_col.iloc[:, -1]*self.p + last_col.iloc[:, -1].shift(periods=-1)*self.q)*np.exp(-self.riskfree*self.dt)
            
            if self.EU != True:
                temp = np.diagonal(np.fliplr(price_diff), i)
                temp = np.append(temp, [np.NaN]*i)
                last_col[i] = np.where(last_col[i] >= temp, last_col[i], temp) 

        last_col = last_col.iloc[:, ::-1]
        
        for i in last_col.index:
            last_col.loc[i, :] = last_col.loc[i, :].shift(periods=-i)
            
        last_col.columns = range(len(last_col))
        self.tree_opt = last_col

        return self
    
    
    
    
    def prettify(self, tree):
        
        """"
        
        Transforms tree into classical form
        
        """       
        
        pretty_tree = pd.DataFrame(columns = tree.columns)
        
        for i in tree.index:
            pretty_tree.loc[i, :] = tree.loc[i, :].shift(periods=i)
            pretty_tree.loc[i+0.5, :] = np.NaN

        pretty_tree = pretty_tree.sort_index().reset_index(drop=True)

        for i, column in enumerate(pretty_tree.columns):
            pretty_tree[column] = pretty_tree[column].shift(periods=(int(len(pretty_tree.index)/2)-i))
        
        pretty_tree = pretty_tree.fillna('')
        
        return pretty_tree
    
    
    def combine(self):
        
        """"
        
        Combines option and asset tree into one
        
        """
        
        opt = self.prettify(self.tree_opt)
        opt.index += 1
        opt.loc[0, :] = 0

        asset = self.prettify(self.tree)
        asset.loc[len(asset), :] = 0
        self.tree_comb = (opt.replace('', 10**(-100)) + asset.replace('', 0)).replace(10**(-100), '').drop(0).reset_index(drop=True)
        
        return self

    
    
    
    def BSM(self, precise = False):
        
        """"
        
        Calculate option's price according to Blasck-Scholes-Merton model
        
        precise: defines the value of n, if False: it uses value given during initialization, otherwise, it it equal to 1 mln
        
        """
        
        if self.EU == False:
            raise Exception('American options cannot be evaluated using Black-Scholes model')
        
        dt_precise = 0 if precise != False else self.dt
        
        self.d1 = (np.log(self.asset_price/self.strike) + (self.riskfree + self.sigma**2/2)*(self.T - dt_precise))/(self.sigma*np.sqrt(self.T - dt_precise))
        
        self.d2 = self.d1 - self.sigma*np.sqrt(self.T - dt_precise)
        
        if self.__call == True:
            self.BSM_price = norm.cdf(self.d1)*self.asset_price - norm.cdf(self.d2)*self.strike*np.exp(-1*self.riskfree*(self.T-dt_precise))
        else:
            self.BSM_price = norm.cdf(-self.d2)*self.strike*np.exp(-1*self.riskfree*(self.T-dt_precise)) - norm.cdf(-self.d1)*self.asset_price 

        return self.BSM_price
    
    
    
    def get_net(self, m = 30, width = None):
        
        if width == None:
            width = self.__n
        net = pd.DataFrame()
        
        max_col = list(self.tree_opt.iloc[0, :]).index(0) #if self.__call == False else list(self.tree_opt.iloc[:, 0]).index(0)
        s_max = self.tree.iloc[0, max_col]
    
        self.dS = self.asset_price/m
        k = ceil(s_max/self.dS)
        j = np.arange(k+1)[::-1]

        ###########  Columns ############

        first_col = []
        for i in j:
            first_col.append(i*self.dS)
        
        first_col = np.array(first_col)
        
        # price_diff = first_col - self.strike if self.__call == True else self.strike - first_col ####????????
        
        price_diff = self.strike - first_col    

        last_col = np.where(price_diff>0, price_diff, 0) 


        ###########  Rows ############

        first_row = ['Share Price/ Time']
        for i in range(width+1):
            time = round(i*self.T/width, 5)
            first_row.append(time)
        
        
        ###########  ABC ############
        
        div = 0 if self.__divs==False else float(input('enter dividend yield: '))
        
        abc = [0, 1 , 0]

        for num in j[1:-1]:    
            aj = 0.5*(self.riskfree-div)*num*self.dt - 0.5*self.sigma**2*num**2*self.dt

            bj = 1 + self.sigma**2*num**2*self.dt + self.riskfree*self.dt

            cj = -0.5*(self.riskfree-div)*num*self.dt - 0.5*self.sigma**2*num**2*self.dt

            abc = np.vstack([abc, [aj, bj, cj]])

        self.abc = np.vstack([abc, [0, 1, 0]]).T

        zeroes = np.zeros([len(last_col) - self.abc.shape[0], self.abc.shape[1]])
        abc_matrix = np.vstack([self.abc, zeroes])

        for i in range(len(abc_matrix)):
            abc_matrix[:, i] = np.roll(abc_matrix[:, i], i-1)

        inv = np.linalg.inv(abc_matrix)
        
        ###########  Grid ############
            
        net = pd.DataFrame()
        net[0] = last_col

        for i in range(1, width+1):
            net[i] = net.iloc[:, -1] @ inv
            
            if self.EU != True:
                net[i] = np.where(net[i]>last_col, net[i], last_col)
        
        net['Share price'] = first_col
        net = net.iloc[:, ::-1]
        net.columns = first_row
        
        return net
        
    def params(self):
        return self.__dict__

In [11]:
yndx = Option(473.4, 540, 0.4676, start = '26/06/2023', end = '16/08/2023', EU = 1, call = 1, n=10)

In [12]:
yndx.BSM(True)

12.680578847768288

In [22]:
yndx.grow_tree().prettify(yndx.tree_opt)

,0,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,,
1,,,,,,,,,,,282.765191
2,,,,,,,,,,239.08805,
3,,,,,,,,,197.789391,,196.658795
4,,,,,,,,158.741284,,157.611872,
5,,,,,,,122.548449,,120.69445,,119.563854
6,,,,,,90.877537,,87.232561,,84.662575,
7,,,,,64.912371,,59.939994,,54.527251,,50.537275
8,,,,44.847271,,39.528509,,33.241213,,25.023328,
9,,,30.098622,,25.220336,,19.550085,,12.3902,,0.0


# dfghjk

In [11]:
exam = Option(46672, 50000, 0.255, start = '29/06/2023', end = '19/07/2023', EU = True, call = False, n=50)

In [12]:
exam.BSM()

3323.437884670835

In [13]:
a = pd.Series(exam.params())

In [14]:
a = a.drop(['_Option__call', '_Option__n', '_Option__divs'])

In [15]:
a['n'] = 50
a['Type'] = 'Put'
a['Dividend yield'] = 0.0792

In [16]:
a

asset_price                     46672
strike                          50000
sigma                           0.255
EU                               True
riskfree                        0.075
start             2023-06-29 00:00:00
end               2023-07-19 00:00:00
days                               20
T                            0.054795
dt                           0.001096
d1                          -1.067933
d2                          -1.127024
BSM_price                 3323.437885
n                                  50
Type                              Put
Dividend yield                 0.0792
dtype: object

In [17]:
exam.grow_tree()

In [21]:
exam.grow_tree().get_net()

enter dividend yield:  0.12


,Share Price/ Time,0.0,0.0011,0.00219,0.00329,0.00438,0.00548,0.00658,0.00767,0.00877,...,0.04493,0.04603,0.04712,0.04822,0.04932,0.05041,0.05151,0.0526,0.0537,0.05479
0,60673.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,59117.866667,3.550436,3.298121,3.057348,2.827981,2.609874,2.402862,2.206768,2.021398,1.846545,...,0.002324,0.001392,0.000786,0.000412,0.000196,0.000082,0.000029,0.000008,0.000001,0.000000
2,57562.133333,12.233426,11.490538,10.773835,10.083332,9.419016,8.780845,8.168748,7.582622,7.022332,...,0.023186,0.014841,0.009004,0.005108,0.002656,0.001227,0.000478,0.000142,0.000025,0.000000
3,56006.400000,36.788806,34.988305,33.228935,31.511437,29.836536,28.204942,26.617343,25.074405,23.576765,...,0.219229,0.150771,0.098954,0.061218,0.035057,0.018062,0.007973,0.002736,0.000563,0.000000
4,54450.666667,102.129346,98.318328,94.547682,90.819233,87.134864,83.496512,79.906175,76.365905,72.877808,...,1.934938,1.436434,1.025830,0.697295,0.444055,0.258350,0.131407,0.053419,0.013558,0.000000
5,52894.933333,259.378085,252.524502,245.667908,238.809938,231.952354,225.097050,218.246066,211.401594,204.565992,...,15.375250,12.373874,9.671006,7.279431,5.210936,3.475917,2.082895,1.037942,0.343988,0.000000
6,51339.200000,593.542512,583.580591,573.529943,563.388894,553.155750,542.828799,532.406317,521.886572,511.267831,...,103.788779,90.776448,77.965766,65.413609,53.185656,41.357662,30.016899,19.263819,9.213939,0.000000
7,49783.466667,1205.548808,1194.659512,1183.635979,1172.473621,1161.167582,1149.712718,1138.103576,1126.334364,1114.398927,...,538.756784,509.968178,479.783426,448.073740,414.695981,379.490925,342.281302,302.869593,261.035538,216.533333
8,48227.733333,2152.441492,2144.319296,2136.143574,2127.914157,2119.630943,2111.293912,2102.903136,2094.458788,2085.961162,...,1800.683606,1794.466844,1788.790209,1783.732724,1779.383525,1775.843148,1773.224987,1771.656941,1771.283263,1772.266667
9,46672.000000,3390.487117,3386.837350,3383.223418,3379.647681,3376.112619,3372.620838,3369.175076,3365.778207,3362.433248,...,3310.367387,3311.603315,3313.044830,3314.686377,3316.519592,3318.532788,3320.710374,3323.032172,3325.472645,3328.000000


In [78]:
eu.to_excel('eu_net.xlsx')